# Haiku Level 0 Data Scientist Experience - Chapter 4
## Part 2 - New account registration and code execution requests

Link to the original Haiku tutorial: https://dm-haiku.readthedocs.io/en/latest/notebooks/build_your_own_haiku.html

In [ ]:
# Import the necessary libraries
import syft as sy
sy.requires(">=0.8-beta")
import jax
import jax.numpy as jnp
import haiku as hk

In [ ]:
# Register a client to the domain
node = sy.orchestra.launch(name="test-domain-1")
guest_domain_client = node.client
guest_domain_client.register(name="Jane Doe", email="jane@caltech.edu", password="abc123", institution="Caltech", website="https://www.caltech.edu/")
guest_domain_client.login(email="jane@caltech.edu", password="abc123")

In [ ]:
!pip install dm-tree graphviz

In [ ]:
# Create a function for code execution
# ATTENTION: ALL LIBRARIES USED SHOULD BE DEFINED INSIDE THE FUNCTION CONTEXT!!!

@sy.syft_function(input_policy=sy.ExactMatch(),
                  output_policy=sy.SingleExecutionExactOutput())
def tabulate_example():
    import jax
    import jax.numpy as jnp
    import haiku as hk
    
    def f(x):
        return hk.nets.MLP([300, 100, 10])(x)

    f = hk.transform(f)
    x = jnp.ones([8, 28 * 28])

    print(hk.experimental.tabulate(f)(x))
    
    for method_invocation in hk.experimental.eval_summary(f)(x):
        print(method_invocation)
        
@sy.syft_function(input_policy=sy.ExactMatch(),
                  output_policy=sy.SingleExecutionExactOutput())
def graphviz_example():
    import jax
    import jax.numpy as jnp
    import haiku as hk
    
    def f(a):
        b = jnp.sin(a)
        c = jnp.cos(b)
        d = b + c
        e = a + d
        return e

    x = jnp.ones([1])
    dot = hk.experimental.to_dot(f)(x)

    import graphviz
    return graphviz.Source(dot)
    
    # def f(x):
    #     return hk.nets.MLP([300, 100, 10])(x)

    # f = hk.transform(f)

    # rng = jax.random.PRNGKey(42)
    # x = jnp.ones([8, 28 * 28])
    # params = f.init(rng, x)

    # dot = hk.experimental.to_dot(f.apply)(params, None, x)
    # graphviz.Source(dot)

In [ ]:
# Test our function locally 
graphviz_example()

In [ ]:
# Submit the function for code execution
guest_domain_client.api.services.code.request_code_execution(first_example)

In [ ]:
guest_domain_client.api.services.code.first_example()

### Go to the Data Owner Notebook for Part 2!

## Part 3 - Downloading the Results

In [ ]:
guest_domain_client._api = None
_ = guest_domain_client.api

In [ ]:
result = guest_domain_client.api.services.code.haiku_nets_example()

In [ ]:
result.get_result()

In [ ]:
print(result.get_stderr())